# <u>`Project 5 Part 1 Core`
* Authored By: Eric N. Valdez
* Date: 2/29/24

# <u><center> [Metabolic Syndrome Prediction](https://data.world/informatics-edu/metabolic-syndrome-prediction)
* The dataset for analysis came from the [NHANES](https://www.cdc.gov/nchs/nhanes/index.htm) initiative where the following variables were combined from multiple tables with SQL: abnormal waist circumference, triglycerides above 150, HDL cholesterol below 50 in women or 40 in men, history of hypertension and mildly elevated fasting blood sugar (100-125). Numerous other variables were added, such as uric acid, race, income, etc. that might contribute to the model but we will not sure, until we test the model.
* [Data Dictionary](https://docs.google.com/document/d/11D8SRng4m7w3ONQkX0bm6_3etsyjD7CDJCNZBLtprzk) as listed below

# <u>Assignment:

#### When choosing your dataset, consider the following:
* What is the target? (You are required to complete a classification task for this project)
* What does one row represent? (A person? A business? An event? A product?)
* How many features does the data have?
* How many rows are in the dataset?
* What opportunities exist for dimensionality reduction or feature selection with this dataset?
* What, if any, challenges do you foresee in cleaning, exploring, or modeling this dataset?

# <u>For this task:
* Choose the a dataset from the provided list

# <u>Imports:

# <u> Custom Functions:

# <u>Load Data:

# `Explore/clean the data`

# `Exploratory Visualizations`
* Creating explorator visualizations to understand your data and search for trends.

# `Choose a model`
* ## Preprocess data

* ## Fit and evaluate a default model

* ## Extract and visualize the top 10 features using permutation importance
    * (from Intro to ML Week 4)

* ## Add your observations in a Markdown:
    * ### Do these features make sense based on the business case?

* ## `Create Explanatory Visualizations for the most important features`
    *  ### Select 2 out of the top 10 features from your permutation importances and produce explanatory visualizations showing the relationship between the feature and the target.
    *  ### The purpose is to demonstrate key trends you found that will be of interest to a stakeholder.

## `Explanatory Visualization 1`

## `Explanatory Visualization 2`

## `1. These visuals should be reporting-quality with titles, labels, and a short explanation of the trend.` 
* ### Explain in a text cell the insight associated with each visual.
    * Both of these visualizations should be easily understood by a non-technical audience
        * (Neither of these should be histograms, boxplots, or correlation plots).
## `2. These will be added to your readme in part 4`

## `Explanatory Visualization 1`

## `Explanatory Visualization 2`